In [ ]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import random
import os

**Create a data frame for Yearly report of Florida mental health provider data countywise from year 16-17 to year 22-23**

In [ ]:
# Read Ten years Florida provider report from excel and create a dataframe called as ten_year_report
ten_year_report = pd.read_excel('../FL_mental_health_providers/data/TenYrsReport.xlsx', skiprows=2)
ten_year_report

In [ ]:
# Drop the count and blank columns
ten_year_report = ten_year_report.drop(columns=['Count','Count.1','Count.2','Count.3','Count.4','Count.5','Count.6','Count.7','Rate.7','Count.8','Rate.8','Count.9','Rate.9'], axis=1)#,'Unnamed: 21','Unnamed: 22'], axis=1)
#ten_year_report

In [ ]:
# Rename Rate column with appropriate column names of ten_year_report dataframe 
ten_year_report.columns = ['County','FY_22_23','FY_21_22','FY_20_21','FY_19_20','FY_18_19','FY_17_18','FY_16_17']
#ten_year_report

In [ ]:
# Reorder columns from the year 16-17 to year 22-23
ten_year_report = ten_year_report.reindex(columns=['County','FY_16_17','FY_17_18','FY_18_19','FY_19_20','FY_20_21','FY_21_22','FY_22_23'])
#ten_year_report = ten_year_report.reindex(columns={'County':'County','Rate.6':'FY_16_17', 'Rate.5': 'FY_17_18', 'Rate.4': 'FY_18_19','Rate.3':'FY_19_20','Rate.2':'FY_20_21','Rate.1':'FY_21_22','Rate':'FY_22_23'})
ten_year_report

In [ ]:
# Convert the year labels into a dataframe column named "Year"
ten_year_report = ten_year_report.melt(id_vars=["County"]
        , var_name="Year")
ten_year_report

**Convert Fiscal year into datetime column**

In [ ]:
# Create a dictionary with fiscal year values to be replaced with relevant date string
fiscal_year_todate = {'FY_16_17': '06-30-2017'
                     , 'FY_17_18': '06-30-2018'
                     , 'FY_18_19': '06-30-2019'
                     , 'FY_19_20': '06-30-2020'
                     , 'FY_20_21': '06-30-2021'
                     , 'FY_21_22': '06-30-2022'
                     , 'FY_22_23': '06-30-2023'}

# Replace fiscal year with relevant date string using fiscal_year_todate dictionary
ten_year_report['Year'].replace(fiscal_year_todate, inplace=True)
# Convert Year column into date format.
ten_year_report['Year'] = pd.to_datetime(ten_year_report['Year'])
ten_year_report

In [ ]:
# #Below code only works if Year column contains string in any format of date
# from dateutil.parser import parse
# ten_year_report["Year"] = ten_year_report["Year"].apply(lambda x: parse(x))

**Save newly converted yearly report for dashboard**

In [ ]:
# fetch the current working directory
cwd = os.getcwd() 
# select the dashboard folder to save the csv file
dashboard_path = cwd + "\\dashboard" 
# define the file name for csv
filename = 'FL_yearly_report.csv'
# Create the full file path
csv_path = os.path.join(dashboard_path, filename)

In [ ]:
# convert the DataFrame to CSV file and save it in given path
ten_year_report.to_csv(csv_path, index=False) 

**Create a function that brings in all the excel files and then append all the excels together to create one master dataframe**

In [ ]:
# select the data folder to fetch excel files
data_path = cwd + "\\data" 
# list all the file names present in the path
files = os.listdir(data_path)
files

In [ ]:
# Retrieve only monthly excel files 
files_xls = [f for f in files if f[-12:] == 'Monthly.xlsx']
files_xls

In [ ]:
# check what is the result of file path
for f in files_xls:
    file_path = '../FL_mental_health_providers/data/' + f 
file_path # result = '../FL_mental_health_providers/data/EHR_GEHRIMED_C4631G6561_2024-12-31_Facility and Provider GDR Report_Monthly.xlsx'

In [ ]:
# loop over list of monthly excel files to append to empty dataframe
data_list = []
for f in files_xls:
    data = pd.read_excel('../FL_mental_health_providers/data/' + f, 'Provider MTD')
    data.insert(0, 'Date', pd.to_datetime(f.split('_')[3][:7]) , True)
    data_list.append(data)

In [ ]:
# Copy the data from list to dataframe and create a master dataset for provider information
df_provider_2024 = pd.concat(data_list)

In [ ]:
# display the values of master dataframe df_provider_2024
df_provider_2024

In [ ]:
df_provider_2024['Date'].value_counts()

In [ ]:
# # This function assigns the sequential ids to unique names - to be used in masking the data.
# def assign_ids(df, column_name):
#     unique_names = df[column_name].unique() # Get the unique values of 'column_name' column
#     name_to_id = {name: i for i, name in enumerate(unique_names)}
#     df['Provider masked name'] = df[column_name].map(name_to_id)
#     return df

**Create a new column "New Provider Name" which will hold the masked provider names and place it as 2nd column of dataframe**

In [ ]:
# Create an array with unique values of provider name column
unique_names = df_provider_2024['Provider Name'].unique() 
# loop over to create unique masked ids
name_to_id = {name: i for i, name in enumerate(unique_names)} 

In [ ]:
# # map and assign newly created id in new column of dataframe
# df_provider_2024['Provider masked name'] = df_provider_2024['Provider Name'].map(name_to_id) 

In [ ]:
# Using insert() to add a new column at position 1 (2nd column) and assign newly created ids by mapping the values to it
df_provider_2024.insert(1, "New_Provider_Name", df_provider_2024['Provider Name'].map(name_to_id) , True)

In [ ]:
# prefix 'Provider' string to the value of masked column
df_provider_2024['New_Provider_Name'] = 'Provider-' + df_provider_2024['New_Provider_Name'].astype(str)

In [ ]:
#df_provider_2024 = assign_ids(df_provider_2024, 'Provider Name')
df_provider_2024

In [ ]:
# Drop the sensitive column with provider name columns=['B', 'C']
df_provider_2024 = df_provider_2024.drop(columns=['Unnamed: 0','Provider Name'],axis=1)
df_provider_2024

**Save the newly converted file**

In [ ]:
filename = 'Provider_data_2024.csv'
# Create the full file path
csv_path = os.path.join(dashboard_path, filename)
# convert the DataFrame to CSV file and save it in given path
df_provider_2024.to_csv(csv_path, index=False) 

**Create a monthly total dataframe**

In [ ]:
df_monthly_total = df_provider_2024[df_provider_2024['New_Provider_Name']=='Provider-100']

In [ ]:
df_monthly_total.head()

In [ ]:
filename = 'monthly_total.csv'
# Create the full file path
csv_path = os.path.join(dashboard_path, filename)
# convert the DataFrame to CSV file and save it in given path
df_monthly_total.to_csv(csv_path, index=False) 

**Delete rows representing total and create master dataframe**

In [ ]:
df_provider_2024

## Check the code - delete is not working properly

In [ ]:
df_master_data_2024 = df_provider_2024.drop(df_provider_2024[(df_provider_2024.New_Provider_Name == 'Provider-100')].index)
df_master_data_2024

In [ ]:
# filename = 'master_data_2024.csv'
# # Create the full file path
# csv_path = os.path.join(dashboard_path, filename)
# # convert the DataFrame to CSV file and save it in given path
# df_master_data_2024.to_csv(csv_path, index=False) 